In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

Data Loading

In [2]:
ratings_train = pd.read_csv("../Data_preprocess/Kaggle/ratios/edge_80.csv",low_memory=False)
ratings_test = pd.read_csv("rate_test.csv",low_memory=False)

In [3]:
datasets_train = ratings_train.dataset_id.unique()
model_train = ratings_train.model_id.unique()
datasets_test = ratings_test.dataset_id.unique()
model_test = ratings_test.model_id.unique()

Rating Matrix

In [4]:
data_model_train_matrix = pd.DataFrame(index=datasets_train,columns=model_train)
data_model_test_matrix = pd.DataFrame(index=datasets_test,columns=model_test)

In [5]:
for row in ratings_train.itertuples():
    data_model_train_matrix[row[2]][row[1]] = row[3]
data_model_train_matrix = data_model_train_matrix.fillna(0)

In [6]:
for row in ratings_test.itertuples():
    data_model_test_matrix[row[2]][row[1]] = row[3]
data_model_test_matrix = data_model_test_matrix.fillna(0)

Dataset Similarity Matrix

In [7]:
dataset_similarity = cosine_similarity(data_model_train_matrix)
model_similarity = cosine_similarity(data_model_train_matrix.T)
dataset_similarity_csv = pd.DataFrame(dataset_similarity,index=datasets_train,columns=datasets_train)

In [8]:
dataset_similarity_csv

,48,50,57,33,8,56,12,37,51,23,...,25,58,3,66,24,59,42,44,46,41
48,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
50,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.696271,0.000000,0.0,0.0,0.0,0.0,0.732976
57,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.241288,0.0,0.0,0.0,0.0,0.000000
33,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.832734,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
8,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.000000
42,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.000000
44,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.766378,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.0,0.000000
46,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,0.000000


Prediction

In [9]:
def predict(ratings, similarity):
        mean_user_rating = ratings.mean(axis=1)
        ratings_diff = ratings - np.array(mean_user_rating)[:,np.newaxis]
        pred = np.array(mean_user_rating)[:,np.newaxis] + np.dot(similarity,ratings_diff) / np.array([np.abs(similarity).sum(axis = 1)]).T
        return pred

In [10]:
train_prediction = predict(data_model_train_matrix,dataset_similarity_csv)
model_prediction_train = pd.DataFrame(train_prediction,index=datasets_train,columns=model_train).sort_index().sort_index(axis=1)
model_prediction_test = pd.DataFrame(index=datasets_test,columns=model_test)

In [11]:
model_prediction_train

,72,73,74,75,76,77,78,79,80,81,...,1862,1863,1864,1865,1866,1867,1868,1869,1870,1871
0,0.482122,0.556812,0.463015,0.726297,0.592800,0.661620,0.337892,0.490400,0.612022,0.441616,...,0.003876,0.003876,0.003876,0.003876,0.003876,0.003876,0.003876,0.003876,0.003876,0.003876
2,0.002394,0.002394,0.002394,0.002394,0.002394,0.002394,0.002394,0.002394,0.002394,0.002394,...,0.002394,0.002394,0.002394,0.002394,0.002394,0.002394,0.002394,0.002394,0.002394,0.002394
3,-0.000797,-0.000797,-0.000797,-0.000797,-0.000797,-0.000797,-0.000797,-0.000797,-0.000797,-0.000797,...,0.105809,0.129497,0.115202,0.096343,0.101779,0.155240,0.083820,0.186908,0.230492,0.122938
4,-0.001813,-0.001813,-0.001813,-0.001813,-0.001813,-0.001813,-0.001813,-0.001813,-0.001813,-0.001813,...,-0.001813,-0.001813,-0.001813,-0.001813,-0.001813,-0.001813,-0.001813,-0.001813,-0.001813,-0.001813
5,-0.000920,-0.000920,-0.000920,-0.000920,-0.000920,-0.000920,-0.000920,-0.000920,-0.000920,-0.000920,...,-0.000920,-0.000920,-0.000920,-0.000920,-0.000920,-0.000920,-0.000920,-0.000920,-0.000920,-0.000920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,-0.000250,-0.000250,-0.000250,-0.000250,-0.000250,-0.000250,-0.000250,-0.000250,-0.000250,-0.000250,...,-0.000250,-0.000250,-0.000250,-0.000250,-0.000250,-0.000250,-0.000250,-0.000250,-0.000250,-0.000250
68,0.374067,0.476277,0.374647,0.654693,0.506144,0.583948,0.210180,0.394586,0.527765,0.323594,...,-0.003030,-0.003030,-0.003030,-0.003030,-0.003030,-0.003030,-0.003030,-0.003030,-0.003030,-0.003030
69,-0.001426,-0.001426,-0.001426,-0.001426,-0.001426,-0.001426,-0.001426,-0.001426,-0.001426,-0.001426,...,-0.001426,-0.001426,-0.001426,-0.001426,-0.001426,-0.001426,-0.001426,-0.001426,-0.001426,-0.001426
70,-0.000752,-0.000752,-0.000752,-0.000752,-0.000752,-0.000752,-0.000752,-0.000752,-0.000752,-0.000752,...,-0.000752,-0.000752,-0.000752,-0.000752,-0.000752,-0.000752,-0.000752,-0.000752,-0.000752,-0.000752


Metadata Based

In [12]:
meta_datasets = pd.read_csv("datasets_v.csv",low_memory=False)
datasets = meta_datasets.data_id.unique()
meta_datasets = meta_datasets.loc[:,('v1','v2','v3','v4','v5','v6','v7','v8')]

In [13]:
meta_dataset_similarity = cosine_similarity(meta_datasets.values.tolist())
meta_dataset_similarity = pd.DataFrame(meta_dataset_similarity)

In [14]:
i, j = 0, 0
while i < len(datasets):
    while j < len(datasets):
        meta_dataset_similarity[i][j] = None
        i += 1
        j += 1

In [15]:
def Find_Top_k(i,sim_matrix,k):
    row = sim_matrix.loc[i]
    row = row.sort_values(ascending=False)
    index_row = row.index
    index_row = index_row.values.tolist()
    return index_row[:k]

In [16]:
Find_Top_k(1,meta_dataset_similarity,5)

[22, 15, 65, 61, 19]

In [17]:
def find_sim_index(index):
    row1 = meta_dataset_similarity.loc[index]
    row1_max_index = row1[row1 == row1.max()].index[0]
    return row1_max_index

In [18]:
model_prediction_train

,72,73,74,75,76,77,78,79,80,81,...,1862,1863,1864,1865,1866,1867,1868,1869,1870,1871
0,0.482122,0.556812,0.463015,0.726297,0.592800,0.661620,0.337892,0.490400,0.612022,0.441616,...,0.003876,0.003876,0.003876,0.003876,0.003876,0.003876,0.003876,0.003876,0.003876,0.003876
2,0.002394,0.002394,0.002394,0.002394,0.002394,0.002394,0.002394,0.002394,0.002394,0.002394,...,0.002394,0.002394,0.002394,0.002394,0.002394,0.002394,0.002394,0.002394,0.002394,0.002394
3,-0.000797,-0.000797,-0.000797,-0.000797,-0.000797,-0.000797,-0.000797,-0.000797,-0.000797,-0.000797,...,0.105809,0.129497,0.115202,0.096343,0.101779,0.155240,0.083820,0.186908,0.230492,0.122938
4,-0.001813,-0.001813,-0.001813,-0.001813,-0.001813,-0.001813,-0.001813,-0.001813,-0.001813,-0.001813,...,-0.001813,-0.001813,-0.001813,-0.001813,-0.001813,-0.001813,-0.001813,-0.001813,-0.001813,-0.001813
5,-0.000920,-0.000920,-0.000920,-0.000920,-0.000920,-0.000920,-0.000920,-0.000920,-0.000920,-0.000920,...,-0.000920,-0.000920,-0.000920,-0.000920,-0.000920,-0.000920,-0.000920,-0.000920,-0.000920,-0.000920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,-0.000250,-0.000250,-0.000250,-0.000250,-0.000250,-0.000250,-0.000250,-0.000250,-0.000250,-0.000250,...,-0.000250,-0.000250,-0.000250,-0.000250,-0.000250,-0.000250,-0.000250,-0.000250,-0.000250,-0.000250
68,0.374067,0.476277,0.374647,0.654693,0.506144,0.583948,0.210180,0.394586,0.527765,0.323594,...,-0.003030,-0.003030,-0.003030,-0.003030,-0.003030,-0.003030,-0.003030,-0.003030,-0.003030,-0.003030
69,-0.001426,-0.001426,-0.001426,-0.001426,-0.001426,-0.001426,-0.001426,-0.001426,-0.001426,-0.001426,...,-0.001426,-0.001426,-0.001426,-0.001426,-0.001426,-0.001426,-0.001426,-0.001426,-0.001426,-0.001426
70,-0.000752,-0.000752,-0.000752,-0.000752,-0.000752,-0.000752,-0.000752,-0.000752,-0.000752,-0.000752,...,-0.000752,-0.000752,-0.000752,-0.000752,-0.000752,-0.000752,-0.000752,-0.000752,-0.000752,-0.000752


In [19]:
for i in datasets_test:
        max_index = find_sim_index(i)
        model_prediction_test.loc[i] = model_prediction_train.loc[max_index]
model_prediction_test = model_prediction_test.sort_index(axis=1)

KeyError: 1

In [ ]:
data_model_test_matrix.sort_index(axis=1)

In [ ]:
datasets_test.tolist()

In [ ]:
for i in datasets_test.tolist():
    for j in model_test.tolist():
        if data_model_test_matrix.loc[i][j] == 0:
            model_prediction_test.loc[i][j] = None

In [ ]:
model_prediction_test.sort_index(axis=1)

In [ ]:
new_result = pd.DataFrame(columns={"dataset","model","predict_balance_accuracy","groundtruth_balance_accuracy"})

In [ ]:
for i in datasets_test:
    for j in model_test:
        if model_prediction_test.loc[i][j] is not None:
            predict_balance_accuracy = model_prediction_test.loc[i][j]
            groundtruth_balance_accuracy = data_model_test_matrix.loc[i][j]
            new_result = new_result.append([{'dataset':i,'model':j,'predict_balance_accuracy':predict_balance_accuracy,'groundtruth_balance_accuracy':groundtruth_balance_accuracy}],ignore_index=True)

In [ ]:
new_result

In [ ]:
# new_result.to_csv("Kaggle_result",sep='\t')